## Let try to build a basic Ai agent ;; at intial it will only search the web if it feels like it has to search something on web

In [ ]:
import os
os.environ["OPENAI_API_KEY"]= ""


In [17]:
from IPython.display import display, HTML

display(HTML("<style>pre {white-space: pre-wrap;}</style>"))


In [18]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [4]:
!pip uninstall -y langchain langchain-core langchain-community langchain-openai


Found existing installation: langchain 1.2.0
Uninstalling langchain-1.2.0:
  Successfully uninstalled langchain-1.2.0
Found existing installation: langchain-core 1.2.1
Uninstalling langchain-core-1.2.1:
  Successfully uninstalled langchain-core-1.2.1


In [5]:
!pip install -U \
  langchain==1.2.0 \
  langchain-core>=1.2.1 \
  langchain-community>=1.2.1 \
  langchain-openai>=0.2.0 \
  duckduckgo-search

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [6]:
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.4 MB/s eta 0:00:00


In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

In [20]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool= DuckDuckGoSearchRun()

results= search_tool.invoke("top news in india today")
results

'Reuters.com is your online source for the latest India news stories and current events, ensuring our readers up to date with any breaking news developments Get the latest updates and breaking news from India , including politics, elections, government, business, technology, and Bollywood. Live News : Check our live blog for real-time updates on trending news , breaking news from India , World, Sports, entertainment, politics and more. The Times of India Covers all latest breaking news ... Latest News | Find detailed coverage on Latest News at IndianExpress.com. Read all India news and breaking news today from India on politics, business, entertainment, technology, sports, lifestyle and more at CNBC TV18.'

In [6]:
llm= ChatOpenAI()

## now main code is start

In [10]:
# !pip install -U langchainhub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [7]:
from langchain.agents import create_agent


In [21]:
import langchain
import langchain_core
import langchain_community
import langchain_openai

print(langchain.__version__)
print(langchain_core.__version__)
print(langchain_community.__version__)
# print(langchain_openai.__version__)


1.2.0
1.2.5
0.4.1


## if you want pre-define prompt you can use langchain import hub
## so you have to pull ReAct prompt from langchain hub ;; pull the standard ReAct prompt;; bz we are going to build a react agent;; ReAct means (Reasoning + Action)

In [22]:
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage, HumanMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
  """ Handle tool execution error with custom message """
  try:
    return handler(request)
  except Exception as e:
    # return a custom toolmessage to the model
    return ToolMessage(
        content=f"Tool error: Please check input and try again. ({str(e)})",
        tool_call_id=request.tool_call['id']
    )


In [13]:
# create  ReAct agent manually with the react prompt

agent= create_agent(
    model=llm,
    tools=[search_tool],
    middleware=[handle_tool_errors],
)

results= agent.invoke({"messages":[HumanMessage("can you tell me the population of captial of france?")]})
results

{'messages': [HumanMessage(content='can you tell me the population of captial of france?', additional_kwargs={}, response_metadata={}, id='6bcd032d-f8c3-4734-8795-1deed0cd9069'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CsmwenglyKRFwn323bPvOlRHQobTa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b73c5-a43d-7403-a1d4-4f9edb01acf2-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'population of Paris, France'}, 'id': 'call_PDU2tpjlUHfYShWGMOoP78aN', 'type': 'tool_call'}], usage_metadata={'input_token

## see the autonomous thing you don't have to tell anything every (reasoning and tool execution is happending automatically)

## Now let try to now what is ReAct and how it work

## 1. ReAct

* ReAct (reasong + Acting) is a design pattern used in Ai agent that stands for Reasoning+ Acting. It allows a language model (LLM) to interleave internal reasoning (Thought) with external actions (like tool use) in a structured, multi-step process.

* Instead of generating an answer in one go , the model thinks step by step , deciding what it needs to do next and optionally calling tools (APIs, calculators, web search, etc. ) to help it.

## Thought -> action -> observation(tool's answer)
* this is happended untill we don't get final answer

## ReAct is useful for :
* Multi-step problems
* Tool-augmented tasks (web search, database lookup, etc.)
* Making the agent's reasoning **transparent and auditable**

## It was first introduced in the paper
**ReAct: synergizing reasoning and Acting in language models** (Yao et al, 2022)

* agent_scratchpad or thought trace or log means history of Thought+ action+ observation

## now try to add another tool to our agent

In [15]:
@tool
def get_weather_data(city:str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url= f"https://api.weatherstack.com/current?access_key=41c8df4753ad51566c82f903ef3eeb61&query={city}"
  response= requests.get(url)

  return response.json()

get_weather_data.invoke('patna')

{'request': {'type': 'City',
  'query': 'Patna, India',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Patna',
  'country': 'India',
  'region': 'Bihar',
  'lat': '25.600',
  'lon': '85.117',
  'timezone_id': 'Asia/Kolkata',
  'localtime': '2025-12-31 16:05',
  'localtime_epoch': 1767197100,
  'utc_offset': '5.50'},
 'current': {'observation_time': '10:35 AM',
  'temperature': 17,
  'weather_code': 143,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'],
  'weather_descriptions': ['Haze'],
  'astro': {'sunrise': '06:35 AM',
   'sunset': '05:10 PM',
   'moonrise': '01:54 PM',
   'moonset': '03:02 AM',
   'moon_phase': 'Waxing Gibbous',
   'moon_illumination': 84},
  'air_quality': {'co': '816.85',
   'no2': '5.85',
   'o3': '180',
   'so2': '39.45',
   'pm2_5': '161.55',
   'pm10': '165.65',
   'us-epa-index': '5',
   'gb-defra-index': '5'},
  'wind_speed': 5,
  'wind_degree': 290,
  'wind_dir': 'WNW',
  'pressure': 1011,

In [23]:
agent_new= create_agent(
    model=llm,
    tools=[search_tool, get_weather_data],
    middleware= [handle_tool_errors]
)

results= agent_new.invoke({"messages":HumanMessage("Find the capital of Madhya Pradesh, then find it's current wether condition?")})
results

{'messages': [HumanMessage(content="Find the capital of Madhya Pradesh, then find it's current wether condition?", additional_kwargs={}, response_metadata={}, id='67cdbd35-1f54-4838-adb6-34788097694d'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 122, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Csny6KaOkmVm7EqhMImnCcclm2cth', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7401-ae7f-7ed2-9def-da481e5465c8-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'Capital of Madhya Pradesh'}, 'id': 'call_0cW5xVjbWOuiVSKkIo1fcTT3', 'type': 'tool_call'}, {'name